In [1]:
import tensorflow as tf

a = tf.Variable([1, 1])
b = tf.Variable([1, 1])

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(a))
    print(sess.run(b))
    writer = tf.summary.FileWriter("/tmp/test/3")
    writer.add_graph(sess.graph)

[1 1]
[1 1]


In [5]:
import tensorflow as tf
LOGDIR = "/ddd/cpy/cpy/MNIST/MNIST_data/"
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir=LOGDIR, one_hot=True)

# Define a simple convolutional layer
def conv_layer(input, channels_in, channels_out):
    w = tf.Variable(tf.zeros([5, 5, channels_in, channels_out]))
    b = tf.Variable(tf.zeros([channels_out]))
    conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
    act = tf.nn.relu(conv + b)
    return act


# Define a fully connected layer
def fc_layer(input, channels_in, channels_out):
    w = tf.Variable(tf.zeros([channels_in, channels_out]))
    b = tf.Variable(tf.zeros([channels_out]))
    act = tf.nn.relu(tf.matmul(input, w) + b)
    return act


# Setup placeholders, and reshape the data
# mnist images are 28 x 28 = 784
# output should be 10 to correspond to 10 digits
# need to turn mnist vectors into an image
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])


# create the network
conv1 = conv_layer(x_image, 1, 32)
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

conv2 = conv_layer(pool1, 32, 64)
pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
flattened = tf.reshape(pool2, [-1, 7 * 7 * 64])

fc1 = fc_layer(flattened, 7 * 7 * 64, 1024)
logits = fc_layer(fc1, 1024, 10)

# Compute cross entropy as the loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# Use an AdamOptimizer to train the network
train_step = tf.train.AdadeltaOptimizer(1e-4).minimize(cross_entropy)

# Compute the accuracy
correct_predition = tf.equal(tf.arg_max(logits, 1), tf.arg_max(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predition, tf.float32))


# Initialize the variables
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Train for 2000 steps
for i in range(200):
    batch = mnist.train.next_batch(100)

    # Occasionally report accuracy
    if i % 50 == 0:
        [train_accuracy] = sess.run([accuracy], feed_dict={x: batch[0], y:batch[1]})
        print("step %d, training accuracy %g" % (i, train_accuracy))

    # Run the training step
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

writer = tf.summary.FileWriter("/tmp/mnist_demo/1")
writer.add_graph(sess.graph)

print("finish")


Extracting /ddd/cpy/cpy/MNIST/MNIST_data/train-images-idx3-ubyte.gz
Extracting /ddd/cpy/cpy/MNIST/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /ddd/cpy/cpy/MNIST/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /ddd/cpy/cpy/MNIST/MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.07
step 50, training accuracy 0.1
step 100, training accuracy 0.09
step 150, training accuracy 0.12


In [6]:
import tensorflow as tf

# example from tensorflow dev summit 2017 dandelion mane

LOGDIR = "/ddd/cpy/cpy/MNIST/MNIST_data/"
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir=LOGDIR, one_hot=True)


# Define a simple convolutional layer
def conv_layer(input, channels_in, channels_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, channels_in, channels_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


# Define a fully connected layer
def fc_layer(input, channels_in, channels_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([channels_in, channels_out], stddev=0.1, name="W"))
        b = tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")
        act = tf.matmul(input, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act


# Setup placeholders, and reshape the data
# mnist images are 28 x 28 = 784
# output should be 10 to correspond to 10 digits
# need to turn mnist vectors into an image
x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")
x_image = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image("input", x_image, 3)

# create the network
conv1 = conv_layer(x_image, 1, 32, "conv1")
conv2 = conv_layer(conv1, 32, 64, "conv2")

flattened = tf.reshape(conv2, [-1, 7 * 7 * 64])
fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
logits = fc_layer(fc1, 1024, 10, "fc2")

# Compute cross entropy as the loss function
with tf.name_scope("xent"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name="xent")
    tf.summary.scalar("xent", cross_entropy)

# Use an AdamOptimizer to train the network
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Compute the accuracy
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

# Initialize the variables
sess = tf.Session()
sess.run(tf.global_variables_initializer())

merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("/tmp/mnist_demo/3")
writer.add_graph(sess.graph)

# Train for 2001 steps
for i in range(200):
    batch = mnist.train.next_batch(100)

    # Occasionally report accuracy
    if i % 5 == 0:
        [train_accuracy, s] = sess.run([accuracy, merged_summary], feed_dict={x: batch[0], y: batch[1]})
        writer.add_summary(s, i)
        print("step %d, training accuracy %g" % (i, train_accuracy))

    # Run the training step
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

Extracting /ddd/cpy/cpy/MNIST/MNIST_data/train-images-idx3-ubyte.gz
Extracting /ddd/cpy/cpy/MNIST/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /ddd/cpy/cpy/MNIST/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /ddd/cpy/cpy/MNIST/MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.09
step 5, training accuracy 0.15
step 10, training accuracy 0.3
step 15, training accuracy 0.36
step 20, training accuracy 0.46
step 25, training accuracy 0.54
step 30, training accuracy 0.67
step 35, training accuracy 0.66
step 40, training accuracy 0.66
step 45, training accuracy 0.73
step 50, training accuracy 0.74
step 55, training accuracy 0.79
step 60, training accuracy 0.79
step 65, training accuracy 0.72
step 70, training accuracy 0.87
step 75, training accuracy 0.83
step 80, training accuracy 0.87
step 85, training accuracy 0.84
step 90, training accuracy 0.8
step 95, training accuracy 0.81
step 100, training accuracy 0.88
step 105, training accuracy 0.85
step 110, training accuracy 